## Observations and Insights 

In [67]:
%matplotlib widget

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
pd.options.mode.chained_assignment = None 

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
mouse_metadata.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [80]:
study_results = pd.read_csv(study_results_path)
study_results.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [81]:
# Combine the data into a single dataset
combo_d = pd.merge(mouse_metadata, study_results, on="Mouse ID")
# Display the data table for preview
combo_d

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [82]:
# Checking the number of mice.
num_mice = len(combo_d["Mouse ID"].unique())
num_mice_df = pd.DataFrame({"Total Number of Mice":[num_mice]})
num_mice_df

,Total Number of Mice
0,249


In [83]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
grpd_combod = combo_d.groupby(['Mouse ID', 'Timepoint'])
grpd_combod.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [84]:
# Optional: Get all the data for the duplicate mouse ID. 

In [85]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cln_combod = combo_d.drop_duplicates(subset='Mouse ID', keep='last', ignore_index=True)
cln_combod

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,45,22.050126,1
1,s185,Capomulin,Female,3,17,45,23.343598,1
2,x401,Capomulin,Female,16,15,45,28.484033,0
3,m601,Capomulin,Male,22,17,45,28.430964,1
4,g791,Ramicane,Male,11,16,45,29.128472,1
...,...,...,...,...,...,...,...,...
244,z314,Stelasyn,Female,21,28,5,45.934712,0
245,z435,Propriva,Female,12,26,10,48.710661,0
246,z581,Infubinol,Female,24,25,45,62.754451,3
247,z795,Naftisol,Female,13,29,45,65.741070,3


In [86]:
# Checking the number of mice in the clean DataFrame.
cln_combod["Mouse ID"].count()

249

## Summary Statistics

In [87]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method is the most straighforward, creating multiple series and putting them all together at the end.
cln_combod["Drug Regimen"].unique()

array(['Ramicane', 'Capomulin', 'Infubinol', 'Placebo', 'Ceftamin',
       'Stelasyn', 'Zoniferol', 'Ketapril', 'Propriva', 'Naftisol'],
      dtype=object)

In [99]:
combod_mean = cln_combod.groupby("Drug Regimen").mean()["Tumor Volume (mm3)"]
combod_median = cln_combod.groupby("Drug Regimen").median()["Tumor Volume (mm3)"]
combod_var = cln_combod.groupby("Drug Regimen").var()["Tumor Volume (mm3)"]
combod_std = cln_combod.groupby("Drug Regimen").std()["Tumor Volume (mm3)"]
combod_sem = cln_combod.groupby("Drug Regimen").sem()["Tumor Volume (mm3)"]
sum_stats = pd.DataFrame({"Average Tumor Volume (mm3)": combod_mean, "Median Tumor Volume (mm3)": combod_median,
                                        "Variance in Tumor Volume (mm3)": combod_var, 
                                        "Standard Deviation in Tumor Volume (mm3)": combod_std,
                                        "Standard Error in Tumor Volume (mm3)": combod_sem})
sum_stats = sum_stats.sort_index()
sum_stats["Average Tumor Volume (mm3)"] = sum_stats["Average Tumor Volume (mm3)"].map("{:.3f}".format)
sum_stats["Median Tumor Volume (mm3)"] = sum_stats["Median Tumor Volume (mm3)"].map("{:.3f}".format)
sum_stats["Variance in Tumor Volume (mm3)"] = sum_stats["Variance in Tumor Volume (mm3)"].map("{:.3f}".format)
sum_stats["Standard Deviation in Tumor Volume (mm3)"] = sum_stats["Standard Deviation in Tumor Volume (mm3)"].map("{:.3f}".format)
sum_stats["Standard Error in Tumor Volume (mm3)"] = sum_stats["Standard Error in Tumor Volume (mm3)"].map("{:.3f}".format)
sum_stats

,Average Tumor Volume (mm3),Median Tumor Volume (mm3),Variance in Tumor Volume (mm3),Standard Deviation in Tumor Volume (mm3),Standard Error in Tumor Volume (mm3)
Drug Regimen,,,,,
Capomulin,36.668,38.125,32.663,5.715,1.143
Ceftamin,57.754,59.852,69.983,8.366,1.673
Infubinol,58.178,60.165,74.011,8.603,1.721
Ketapril,62.806,64.488,98.921,9.946,1.989
Naftisol,61.206,63.283,106.030,10.297,2.059
Placebo,60.508,62.031,78.760,8.875,1.775
Propriva,56.737,55.841,69.349,8.328,1.666
Ramicane,36.191,36.562,32.166,5.672,1.134
Stelasyn,61.002,62.192,90.332,9.504,1.940


In [89]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
